In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.interpolate import interp1d
from tqdm import tqdm
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore')

In [4]:
import os
os.mkdir('Ground_Water_Level_Stations')

In [ ]:
import zipfile
with zipfile.ZipFile('/content/AIS_2022_03_31.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [5]:
import zipfile
with zipfile.ZipFile('/content/Ground_Water_Level_Stations.zip', 'r') as zip_ref:
    zip_ref.extractall('./Ground_Water_Level_Stations')

In [ ]:
df = pd.read_csv('/content/AIS_2022_03_31.csv')
df.head()

MMSI         BaseDateTime       LAT        LON  SOG    COG  Heading  \
0  367702220  2022-03-31T00:00:01  29.78763  -95.08070  0.1  226.5    340.0   
1  671226100  2022-03-31T00:00:01  25.77626  -80.20320  3.2  143.7    511.0   
2  367767250  2022-03-31T00:00:01  29.31623  -94.78829  4.5  228.1    511.0   
3  338327436  2022-03-31T00:00:03  47.29634 -122.42233  0.0  360.0    511.0   
4  367452810  2022-03-31T00:00:06  29.32824  -94.77391  2.6  319.2    511.0   

       VesselName         IMO CallSign  VesselType  Status  Length  Width  \
0      JOE B WARD         NaN  WDI4808        31.0    12.0    21.0    8.0   
1     RELIANCE II  IMO9221322    5VHS7        79.0     0.0    52.0   12.0   
2          GLEN K         NaN  WDJ3358        52.0     0.0     0.0    0.0   
3        COOL KAT  IMO0000000      NaN        36.0     NaN    15.0    3.0   
4  JOHN W JOHNSON  IMO9602344  WDF4516        60.0     0.0    80.0   19.0   

   Draft  Cargo TransceiverClass  
0    NaN   57.0                A  
1    2.5   70.0                A  
2    0.0   52.0                A  
3    NaN    NaN                B  
4    3.0   60.0                A

In [ ]:
df.shape

(7167046, 17)

In [ ]:
df.isna().sum()

MMSI                      0
BaseDateTime              0
LAT                       0
LON                       0
SOG                       0
COG                       0
Heading                   0
VesselName            17201
IMO                 3368784
CallSign             833343
VesselType              817
Status              1628738
Length               438196
Width               1174077
Draft               4461321
Cargo               1624947
TransceiverClass          0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df = df[['BaseDateTime','MMSI','IMO','LAT','LON','SOG','COG']]

In [ ]:
df.isna().sum()

BaseDateTime    0
MMSI            0
IMO             0
LAT             0
LON             0
SOG             0
COG             0
dtype: int64

In [ ]:
df.shape

(1979165, 7)

In [ ]:
df.to_csv('./modified_ais_data.csv',index=False)

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from shapely.geometry import Point as ShapelyPoint
from tqdm import tqdm
from math import atan2, radians, degrees, sin, cos

def sliding_window_segmentation(data, window_size, step_size, mmsi):
    """
    Perform sliding window segmentation on the given DataFrame.

    Parameters:
    - data: Pandas DataFrame containing the trajectory.
    - window_size: The number of data points in each sliding window.
    - step_size: The step size for the sliding window.
    - mmsi: MMSI of the vessel for this segment.

    Returns:
    - segments: A list of DataFrames, each representing a segmented window.
    """
    segments = []
    for start in range(0, len(data) - window_size + 1, step_size):
        window = data.iloc[start:start + window_size].copy()
        window['MMSI'] = mmsi  # Explicitly set MMSI for each segment
        segments.append(window)
    return segments

def lat_lon_range(df_file):
    """
    Calculate the bounding box for latitude and longitude.

    Parameters:
    - df_file: DataFrame containing 'LAT' and 'LON' columns.

    Returns:
    - min_lat, max_lat, min_lon, max_lon: Integer values defining the bounding box.
    """
    min_lat, max_lat = df_file['LAT'].min(), df_file['LAT'].max()
    min_lon, max_lon = df_file['LON'].min(), df_file['LON'].max()
    min_lat, max_lat, min_lon, max_lon = (
        int(np.floor(min_lat)),
        int(np.ceil(max_lat)),
        int(np.floor(min_lon)),
        int(np.ceil(max_lon)),
    )
    return min_lat, max_lat, min_lon, max_lon

def calculate_initial_bearing(start_lat, start_lon, end_lat, end_lon):
    """
    Calculate the initial bearing (forward azimuth) between two geodesic points.

    Parameters:
    - start_lat, start_lon: Coordinates of the starting point.
    - end_lat, end_lon: Coordinates of the ending point.

    Returns:
    - bearing: Initial bearing in degrees.
    """
    start_lat, start_lon, end_lat, end_lon = map(radians, [start_lat, start_lon, end_lat, end_lon])
    delta_lon = end_lon - start_lon
    x = sin(delta_lon) * cos(end_lat)
    y = cos(start_lat) * sin(end_lat) - sin(start_lat) * cos(end_lat) * cos(delta_lon)
    bearing = atan2(x, y)
    return (degrees(bearing) + 360) % 360  # Normalize to 0-360 degrees

def geodesic_interpolation(start_lat, start_lon, end_lat, end_lon, num_points, water_mask):
    """
    Perform geodesic interpolation between two points while ensuring points remain on water.

    Parameters:
    - start_lat, start_lon: Coordinates of the start point.
    - end_lat, end_lon: Coordinates of the end point.
    - num_points: Number of intermediate points to generate.
    - water_mask: GeoDataFrame containing water polygons.

    Returns:
    - lats, lons: Lists of interpolated latitude and longitude values.
    """
    start_point = ShapelyPoint(start_lon, start_lat)
    total_distance = geodesic((start_lat, start_lon), (end_lat, end_lon)).meters
    bearing = calculate_initial_bearing(start_lat, start_lon, end_lat, end_lon)

    lats, lons = [], []
    spatial_index = water_mask.sindex  # Create a spatial index for efficient lookups
    buffered_water_mask = water_mask.buffer(0.01)  # Apply a small buffer (~1 km)

    for frac in np.linspace(0, 1, num_points):
        distance_to_point = total_distance * frac
        interpolated_point = geodesic(meters=distance_to_point).destination((start_lat, start_lon), bearing)
        lat, lon = interpolated_point.latitude, interpolated_point.longitude

        point = ShapelyPoint(lon, lat)

        # Filter water polygons using spatial index
        possible_matches_index = list(spatial_index.intersection(point.bounds))
        possible_matches = buffered_water_mask.iloc[possible_matches_index]

        if not possible_matches.contains(point).any():
            print(f"Skipping interpolated point on land or near boundary: ({lat}, {lon})")
            continue

        lats.append(lat)
        lons.append(lon)

    return lats, lons

def prepare_ais_data_with_geodesic_interpolation(
    data, bounding_box, window_size, step_size, water_mask, min_resample_interval=900  # 15 minutes in seconds
):
    """
    Prepare AIS data using geodesic interpolation and sliding window segmentation.

    Parameters:
    - data: Pandas DataFrame containing AIS data.
    - bounding_box: Tuple defining the bounding box (min_lat, max_lat, min_lon, max_lon).
    - window_size: The number of data points in each sliding window.
    - step_size: The step size for the sliding window.
    - water_mask: GeoDataFrame containing water polygons.
    - min_resample_interval: Minimum resampling interval in seconds.

    Returns:
    - combined_df: A single DataFrame containing all segmented trajectories.
    """
    min_lat, max_lat, min_lon, max_lon = bounding_box

    # Step 1: Apply bounding box filter
    filtered_data = data[
        (data['LAT'] >= min_lat) & (data['LAT'] <= max_lat) &
        (data['LON'] >= min_lon) & (data['LON'] <= max_lon)
    ]

    # Step 2: Aggregate by MMSI and sort by timestamp
    grouped = filtered_data.groupby('MMSI')
    segments = []
    segment_id = 0

    # Step 3: Process each MMSI group
    for mmsi, group in tqdm(grouped, desc="Processing MMSI", unit="MMSI"):
        group = group.sort_values(by='BaseDateTime')
        group['BaseDateTime'] = pd.to_datetime(group['BaseDateTime'])

        # Step 4: Interpolate data using geodesic interpolation
        resampled_traj = []

        for i in range(len(group) - 1):
            start_row = group.iloc[i]
            end_row = group.iloc[i + 1]

            # Calculate time difference
            time_diff = (end_row['BaseDateTime'] - start_row['BaseDateTime']).total_seconds()

            if time_diff < min_resample_interval:
                print(f"Skipping interpolation for gap < 15 minutes between {start_row['BaseDateTime']} and {end_row['BaseDateTime']}")
                resampled_traj.append((start_row['BaseDateTime'], start_row['LAT'], start_row['LON'], start_row['SOG'], start_row['COG']))
                continue

            num_points = int(time_diff / min_resample_interval) + 1
            print(f"Performing interpolation for gap >= 15 minutes between {start_row['BaseDateTime']} and {end_row['BaseDateTime']} with {num_points} points")

            # Perform geodesic interpolation
            lats, lons = geodesic_interpolation(
                start_row['LAT'], start_row['LON'],
                end_row['LAT'], end_row['LON'],
                num_points, water_mask
            )

            # Interpolate SOG and COG linearly
            sog_interp = np.linspace(start_row['SOG'], end_row['SOG'], len(lats))
            cog_interp = np.linspace(start_row['COG'], end_row['COG'], len(lats))

            # Generate timestamps
            timestamps = pd.date_range(
                start=start_row['BaseDateTime'],
                end=end_row['BaseDateTime'],
                periods=len(lats)
            )

            # Append interpolated points to trajectory
            resampled_traj.extend(zip(timestamps, lats, lons, sog_interp, cog_interp))

        # Ensure the last point is included
        resampled_traj.append((end_row['BaseDateTime'], end_row['LAT'], end_row['LON'], end_row['SOG'], end_row['COG']))

        resampled_df = pd.DataFrame(resampled_traj, columns=['BaseDateTime', 'LAT', 'LON', 'SOG', 'COG'])

        # Perform sliding window segmentation
        trajectory_segments = sliding_window_segmentation(resampled_df, window_size, step_size, mmsi)

        for segment in trajectory_segments:
            segment['SegmentID'] = segment_id
            segments.append(segment)
            segment_id += 1

    # Combine all segmented trajectories into a single DataFrame
    combined_df = pd.concat(segments, ignore_index=True)
    return combined_df

# Example usage
if __name__ == "__main__":
    # Load AIS data
    df = pd.read_csv("/path/to/modified_ais_data.csv")
    df = df[:10000]

    # Load water mask (shapefile with global water boundaries)
    water_mask = gpd.read_file("/path/to/world_shapefile.shp")

    # Calculate bounding box
    min_lat, max_lat, min_lon, max_lon = lat_lon_range(df)
    bounding_box = (min_lat, max_lat, min_lon, max_lon)

    # Define window size and step size
    window_size = 10
    step_size = 5

    # Process AIS data
    segmented_trajectories_df = prepare_ais_data_with_geodesic_interpolation(
        df, bounding_box, window_size, step_size, water_mask, min_resample_interval=900  # 15 minutes
    )

    # Save segmented trajectories to CSV
    segmented_trajectories_df.to_csv("segmented_trajectories.csv", index=False)


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from shapely.geometry import Point as ShapelyPoint
from tqdm import tqdm
from math import atan2, radians, degrees, sin, cos

def sliding_window_segmentation(data, window_size, step_size, mmsi):
    """
    Perform sliding window segmentation on the given DataFrame.

    Parameters:
    - data: Pandas DataFrame containing the trajectory.
    - window_size: The number of data points in each sliding window.
    - step_size: The step size for the sliding window.
    - mmsi: MMSI of the vessel for this segment.

    Returns:
    - segments: A list of DataFrames, each representing a segmented window.
    """
    segments = []
    for start in range(0, len(data) - window_size + 1, step_size):
        window = data.iloc[start:start + window_size].copy()
        window['MMSI'] = mmsi  # Explicitly set MMSI for each segment
        segments.append(window)
    return segments

def lat_lon_range(df_file):
    """
    Calculate the bounding box for latitude and longitude.

    Parameters:
    - df_file: DataFrame containing 'LAT' and 'LON' columns.

    Returns:
    - min_lat, max_lat, min_lon, max_lon: Integer values defining the bounding box.
    """
    min_lat, max_lat = df_file['LAT'].min(), df_file['LAT'].max()
    min_lon, max_lon = df_file['LON'].min(), df_file['LON'].max()
    min_lat, max_lat, min_lon, max_lon = (
        int(np.floor(min_lat)),
        int(np.ceil(max_lat)),
        int(np.floor(min_lon)),
        int(np.ceil(max_lon)),
    )
    return min_lat, max_lat, min_lon, max_lon

def calculate_initial_bearing(start_lat, start_lon, end_lat, end_lon):
    """
    Calculate the initial bearing (forward azimuth) between two geodesic points.

    Parameters:
    - start_lat, start_lon: Coordinates of the starting point.
    - end_lat, end_lon: Coordinates of the ending point.

    Returns:
    - bearing: Initial bearing in degrees.
    """
    start_lat, start_lon, end_lat, end_lon = map(radians, [start_lat, start_lon, end_lat, end_lon])
    delta_lon = end_lon - start_lon
    x = sin(delta_lon) * cos(end_lat)
    y = cos(start_lat) * sin(end_lat) - sin(start_lat) * cos(end_lat) * cos(delta_lon)
    bearing = atan2(x, y)
    return (degrees(bearing) + 360) % 360  # Normalize to 0-360 degrees

def geodesic_interpolation(start_lat, start_lon, end_lat, end_lon, num_points, water_mask):
    """
    Perform geodesic interpolation between two points while ensuring points remain on water.

    Parameters:
    - start_lat, start_lon: Coordinates of the start point.
    - end_lat, end_lon: Coordinates of the end point.
    - num_points: Number of intermediate points to generate.
    - water_mask: GeoDataFrame containing water polygons.

    Returns:
    - lats, lons: Lists of interpolated latitude and longitude values.
    """
    start_point = ShapelyPoint(start_lon, start_lat)
    total_distance = geodesic((start_lat, start_lon), (end_lat, end_lon)).meters
    bearing = calculate_initial_bearing(start_lat, start_lon, end_lat, end_lon)

    lats, lons = [], []
    spatial_index = water_mask.sindex  # Create a spatial index for efficient lookups
    buffered_water_mask = water_mask.buffer(0.01)  # Apply a small buffer (~1 km)

    for frac in np.linspace(0, 1, num_points):
        distance_to_point = total_distance * frac
        interpolated_point = geodesic(meters=distance_to_point).destination((start_lat, start_lon), bearing)
        lat, lon = interpolated_point.latitude, interpolated_point.longitude

        point = ShapelyPoint(lon, lat)

        # Filter water polygons using spatial index
        possible_matches_index = list(spatial_index.intersection(point.bounds))
        possible_matches = buffered_water_mask.iloc[possible_matches_index]

        if not possible_matches.contains(point).any():
            print(f"Skipping interpolated point on land or near boundary: ({lat}, {lon})")
            continue

        lats.append(lat)
        lons.append(lon)

    return lats, lons

def prepare_ais_data_with_geodesic_interpolation(
    data, bounding_box, window_size, step_size, water_mask, min_resample_interval=900  # 15 minutes in seconds
):
    """
    Prepare AIS data using geodesic interpolation and sliding window segmentation.

    Parameters:
    - data: Pandas DataFrame containing AIS data.
    - bounding_box: Tuple defining the bounding box (min_lat, max_lat, min_lon, max_lon).
    - window_size: The number of data points in each sliding window.
    - step_size: The step size for the sliding window.
    - water_mask: GeoDataFrame containing water polygons.
    - min_resample_interval: Minimum resampling interval in seconds.

    Returns:
    - combined_df: A single DataFrame containing all segmented trajectories.
    """
    min_lat, max_lat, min_lon, max_lon = bounding_box

    # Step 1: Apply bounding box filter
    filtered_data = data[
        (data['LAT'] >= min_lat) & (data['LAT'] <= max_lat) &
        (data['LON'] >= min_lon) & (data['LON'] <= max_lon)
    ]

    # Step 2: Aggregate by MMSI and sort by timestamp
    grouped = filtered_data.groupby('MMSI')
    segments = []
    segment_id = 0

    # Step 3: Process each MMSI group
    for mmsi, group in tqdm(grouped, desc="Processing MMSI", unit="MMSI"):
        group = group.sort_values(by='BaseDateTime')
        group['BaseDateTime'] = pd.to_datetime(group['BaseDateTime'])

        # Step 4: Interpolate data using geodesic interpolation
        resampled_traj = []

        for i in range(len(group) - 1):
            start_row = group.iloc[i]
            end_row = group.iloc[i + 1]

            # Calculate time difference
            time_diff = (end_row['BaseDateTime'] - start_row['BaseDateTime']).total_seconds()

            if time_diff < min_resample_interval:
                print(f"Skipping interpolation for gap < 15 minutes between {start_row['BaseDateTime']} and {end_row['BaseDateTime']}")
                resampled_traj.append((start_row['BaseDateTime'], start_row['LAT'], start_row['LON'], start_row['SOG'], start_row['COG']))
                continue

            num_points = int(time_diff / min_resample_interval) + 1
            print(f"Performing interpolation for gap >= 15 minutes between {start_row['BaseDateTime']} and {end_row['BaseDateTime']} with {num_points} points")

            # Perform geodesic interpolation
            lats, lons = geodesic_interpolation(
                start_row['LAT'], start_row['LON'],
                end_row['LAT'], end_row['LON'],
                num_points, water_mask
            )

            # Interpolate SOG and COG linearly
            sog_interp = np.linspace(start_row['SOG'], end_row['SOG'], len(lats))
            cog_interp = np.linspace(start_row['COG'], end_row['COG'], len(lats))

            # Generate timestamps
            timestamps = pd.date_range(
                start=start_row['BaseDateTime'],
                end=end_row['BaseDateTime'],
                periods=len(lats)
            )

            # Append interpolated points to trajectory
            resampled_traj.extend(zip(timestamps, lats, lons, sog_interp, cog_interp))

        # Ensure the last point is included
        resampled_traj.append((end_row['BaseDateTime'], end_row['LAT'], end_row['LON'], end_row['SOG'], end_row['COG']))

        resampled_df = pd.DataFrame(resampled_traj, columns=['BaseDateTime', 'LAT', 'LON', 'SOG', 'COG'])

        # Perform sliding window segmentation
        trajectory_segments = sliding_window_segmentation(resampled_df, window_size, step_size, mmsi)

        for segment in trajectory_segments:
            segment['SegmentID'] = segment_id
            segments.append(segment)
            segment_id += 1

    # Combine all segmented trajectories into a single DataFrame
    combined_df = pd.concat(segments, ignore_index=True)
    return combined_df

# Example usage
if __name__ == "__main__":
    # Load AIS data
    df = pd.read_csv("/path/to/modified_ais_data.csv")
    df = df[:10000]

    # Load water mask (shapefile with global water boundaries)
    water_mask = gpd.read_file("/path/to/world_shapefile.shp")

    # Calculate bounding box
    min_lat, max_lat, min_lon, max_lon = lat_lon_range(df)
    bounding_box = (min_lat, max_lat, min_lon, max_lon)

    # Define window size and step size
    window_size = 10
    step_size = 5

    # Process AIS data
    segmented_trajectories_df = prepare_ais_data_with_geodesic_interpolation(
        df, bounding_box, window_size, step_size, water_mask, min_resample_interval=900  # 15 minutes
    )

